In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, T5Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('trip-data.csv')
df

,id,Activity,location,Description,Type,Price,openig-hours,holiday,act_time,time
0,0,Sharjah Archaeology Museum,Sharjah City,Ancient treasures and artifacts from UAE's pas...,museum,low,Saturday to Thursday:8:00 20:00 Friday:16:00 2...,0,8:00 20:00,"['morning','afternoon','evening']"
1,1,Bait Al Naboodah Museum,Sharjah City,"Historic Gulf home with coral walls, intricate...",museum,low,Saturday to Thursday:8:00 20:00 Friday:16:00 2...,0,8:00 20:00,"['morning','afternoon','evening']"
2,2,Bait Khalid Bin Ibrahim,Sharjah City,Historic house turned boutique hotel offers au...,old_hotel,low,Saturday to Thursday: 8:00 20:00 Friday: 16:00...,0,8:00 20:00,"['morning','afternoon','evening']"
3,3,Sharjah Calligraphy Museumc,Sharjah City,Arab world's only museum dedicated to calligra...,museum,low,Saturday to Thursday: 8:00 20:00 Friday: 16:00...,0,8:00 20:00,"['morning','afternoon','evening']"
4,4,Cultural Square,Sharjah City,Cultural Square: 14-metre high Quran sculpture...,museum,low,friday to friday:12:00 00:00,0,12:00 00:00,"['morning','afternoon','evening','night']"
...,...,...,...,...,...,...,...,...,...,...
1132,1182,Kalba Beach Corniche,East Coast,Explore the Kalba Beach Corniche in East Coast...,trendy,high,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1133,1183,Bait Al Naboodah Museum,Sharjah City,Explore the Bait Al Naboodah Museum in Sharjah...,precious,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1134,1184,Najd Al Maqsar Village,Central Region,Explore the Najd Al Maqsar Village in Central ...,lively,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1135,1185,Kalba Birds of Prey Centre,East Coast,Explore the Kalba Birds of Prey Centre in East...,bustling,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"


In [4]:
df['input_text'] = df['location'] + " "+df["Type"] 
df['target_text'] = df['Description']

In [5]:
df['input_text'].str.len().max()

42

In [6]:
df['target_text'].str.len().max()

163

In [7]:
from datasets import Dataset

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
train_dataset = Dataset.from_pandas(train_df[['input_text', 'target_text']])
test_dataset = Dataset.from_pandas(test_df[['input_text', 'target_text']])

In [10]:
from transformers import T5Tokenizer
import tensorflow as tf
tokenizer = T5Tokenizer.from_pretrained('T5-Generation-Description-Tokenazier')
def tokenize_function(examples):
    input_texts = [str(text) if isinstance(text, str) else "" for text in examples["input_text"]]
    target_texts = [str(text) if isinstance(text, str) else "" for text in examples["target_text"]]
    model_inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=163, return_tensors="tf")
    labels = tokenizer(target_texts, padding="max_length", truncation=True, max_length=163, return_tensors="tf")
    decoder_input_ids = labels['input_ids'][:, :-1]
    decoder_input_ids = tf.concat([tf.zeros_like(decoder_input_ids[:, :1]), decoder_input_ids], axis=1)
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids
    return model_inputs
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
def to_tf_dataset(dataset):
    return tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': dataset['input_ids'],
            'attention_mask': dataset['attention_mask'],
            'decoder_input_ids': dataset['decoder_input_ids'],
        },
        dataset['labels']
    ))


Map:   0%|          | 0/909 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

In [11]:
train_tf_dataset = to_tf_dataset(train_dataset).batch(4)
test_tf_dataset = to_tf_dataset(test_dataset).batch(4)


In [13]:
from keras.optimizers import Adam
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [20]:
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=15)


Epoch 1/15


228/228 [==============================] - 349s 1s/step - loss: 0.3989 - accuracy: 0.9486 - val_loss: 0.1459 - val_accuracy: 0.9747
Epoch 2/15
228/228 [==============================] - 315s 1s/step - loss: 0.1359 - accuracy: 0.9765 - val_loss: 0.1129 - val_accuracy: 0.9798
Epoch 3/15
228/228 [==============================] - 287s 1s/step - loss: 0.1020 - accuracy: 0.9812 - val_loss: 0.1022 - val_accuracy: 0.9814
Epoch 4/15
228/228 [==============================] - 285s 1s/step - loss: 0.0867 - accuracy: 0.9834 - val_loss: 0.0962 - val_accuracy: 0.9822
Epoch 5/15
228/228 [==============================] - 295s 1s/step - loss: 0.0751 - accuracy: 0.9849 - val_loss: 0.0961 - val_accuracy: 0.9828
Epoch 6/15
228/228 [==============================] - 292s 1s/step - loss: 0.0670 - accuracy: 0.9861 - val_loss: 0.0983 - val_accuracy: 0.9831
Epoch 7/15
228/228 [==============================] - 285s 1s/step - loss: 0.0618 - accuracy: 0.9869 - val_loss: 0.0993 - val_accuracy: 0.98

In [63]:
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=15)

Epoch 1/15
228/228 [==============================] - 287s 1s/step - loss: 0.0401 - accuracy: 0.9906 - val_loss: 0.1126 - val_accuracy: 0.9833
Epoch 2/15
228/228 [==============================] - 298s 1s/step - loss: 0.0388 - accuracy: 0.9910 - val_loss: 0.1133 - val_accuracy: 0.9833
Epoch 3/15
228/228 [==============================] - 311s 1s/step - loss: 0.0372 - accuracy: 0.9911 - val_loss: 0.1161 - val_accuracy: 0.9831
Epoch 4/15
228/228 [==============================] - 306s 1s/step - loss: 0.0364 - accuracy: 0.9911 - val_loss: 0.1140 - val_accuracy: 0.9833
Epoch 5/15
228/228 [==============================] - 296s 1s/step - loss: 0.0352 - accuracy: 0.9914 - val_loss: 0.1181 - val_accuracy: 0.9831
Epoch 6/15
228/228 [==============================] - 298s 1s/step - loss: 0.0343 - accuracy: 0.9916 - val_loss: 0.1198 - val_accuracy: 0.9833
Epoch 7/15
228/228 [==============================] - 296s 1s/step - loss: 0.0316 - accuracy: 0.9919 - val_loss: 0.1243 - val_accuracy: 0.9832

In [64]:
model.save_pretrained('T5-Generation-Description-Model')
tokenizer.save_pretrained('T5-Generation-Description-Tokenazier')

('T5-Generation-Description-Tokenazier\\tokenizer_config.json',
 'T5-Generation-Description-Tokenazier\\special_tokens_map.json',
 'T5-Generation-Description-Tokenazier\\spiece.model',
 'T5-Generation-Description-Tokenazier\\added_tokens.json')

In [17]:
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu

In [11]:
def generate_predictions(dataset, model, tokenizer):
    predictions, references = [], []
    for batch in dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=462)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
        predictions.extend(decoded_preds)
        references.extend(decoded_refs)
    return predictions, references

In [19]:
import fractions

# Save the original __new__ method
_original_fraction_new = fractions.Fraction.__new__

def patched_fraction_new(cls, numerator=0, denominator=None, _normalize=True):
    # Call the original __new__ without the _normalize parameter
    if denominator is None:
        return _original_fraction_new(cls, numerator)
    return _original_fraction_new(cls, numerator, denominator)

# Patch the Fraction.__new__ method
fractions.Fraction.__new__ = patched_fraction_new


In [20]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(predictions, references):
    smoothing_function = SmoothingFunction().method1
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        ref_tokens = ref.split()
        pred_tokens = pred.split()
        score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothing_function)
        bleu_scores.append(score)
    return sum(bleu_scores) / len(bleu_scores)

In [21]:
predictions, references = generate_predictions(train_tf_dataset, model, tokenizer)

In [66]:
bleu_score = calculate_bleu(predictions, references)
print(f"Average BLEU Score: {bleu_score}")

Average BLEU Score: 0.6978235164468655


In [26]:
from rouge_score import rouge_scorer

In [24]:
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougeL = 0, 0, 0
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1 += scores['rouge1'].fmeasure
        rouge2 += scores['rouge2'].fmeasure
        rougeL += scores['rougeL'].fmeasure
    n = len(predictions)
    return {
        "rouge1": rouge1 / n,
        "rouge2": rouge2 / n,
        "rougeL": rougeL / n
    }

In [67]:
rouge_scores = calculate_rouge(predictions, references)
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")

ROUGE-1: 0.8111
ROUGE-2: 0.7479
ROUGE-L: 0.8087


In [3]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("T5-Generation-Description-Model")
tokenizer = T5Tokenizer.from_pretrained("T5-Generation-Description-Tokenazier")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at T5-Generation-Description-Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [4]:
new_keywords = ["rsort"]
input_ids = tokenizer(new_keywords, return_tensors="tf").input_ids
generated_ids = model.generate(input_ids, max_length=163, num_beams=5, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)


Explore the Sharjah Ladies Club - Khorfakkan Branch in East Coast. A must-visit destination for visitors.


In [14]:
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=15)

Epoch 1/15


228/228 [==============================] - 331s 1s/step - loss: 0.0227 - accuracy: 0.9936 - val_loss: 0.1445 - val_accuracy: 0.9828
Epoch 2/15
228/228 [==============================] - 299s 1s/step - loss: 0.0218 - accuracy: 0.9938 - val_loss: 0.1440 - val_accuracy: 0.9833
Epoch 3/15
228/228 [==============================] - 306s 1s/step - loss: 0.0201 - accuracy: 0.9941 - val_loss: 0.1468 - val_accuracy: 0.9828
Epoch 4/15
228/228 [==============================] - 301s 1s/step - loss: 0.0195 - accuracy: 0.9941 - val_loss: 0.1481 - val_accuracy: 0.9830
Epoch 5/15
228/228 [==============================] - 292s 1s/step - loss: 0.0195 - accuracy: 0.9943 - val_loss: 0.1421 - val_accuracy: 0.9828
Epoch 6/15
228/228 [==============================] - 290s 1s/step - loss: 0.0178 - accuracy: 0.9946 - val_loss: 0.1483 - val_accuracy: 0.9830
Epoch 7/15
228/228 [==============================] - 301s 1s/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.1462 - val_accuracy: 0.98

In [15]:
model.save_pretrained('T5-Generation-Description-Model-Two')
tokenizer.save_pretrained('T5-Generation-Description-Tokenazier-Two')

('T5-Generation-Description-Tokenazier-Two\\tokenizer_config.json',
 'T5-Generation-Description-Tokenazier-Two\\special_tokens_map.json',
 'T5-Generation-Description-Tokenazier-Two\\spiece.model',
 'T5-Generation-Description-Tokenazier-Two\\added_tokens.json')

In [22]:
bleu_score = calculate_bleu(predictions, references)
print(f"Average BLEU Score: {bleu_score}")

Average BLEU Score: 0.7099501850424708


In [27]:
rouge_scores = calculate_rouge(predictions, references)
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")

ROUGE-1: 0.8146
ROUGE-2: 0.7550
ROUGE-L: 0.8111
